In [ ]:
!export CUDA_VISIBLE_DEVICES=0

In [1]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import time
import os
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from tests import test_prediction, test_generation

In [ ]:
# load all that we need

dataset = np.load('../dataset/wiki.train.npy')
fixtures_pred = np.load('../fixtures/prediction.npz')  # dev
fixtures_gen = np.load('../fixtures/generation.npy')  # dev
fixtures_pred_test = np.load('../fixtures/prediction_test.npz')  # test
fixtures_gen_test = np.load('../fixtures/generation_test.npy')  # test
vocab = np.load('../dataset/vocab.npy')

In [ ]:
def split(dataset):
    dataset_new = []
    for row in dataset:
        found = []
        dots = (row == 79).nonzero()[0]
        if len(dots) > 0:
            for i in dots:
                if i+2 < len(row) and row[i+1] == 1417 and row[i+2] == 1420:
                    found.append(i+2)

        if len(found) == 0:
            dataset_new.append(row)
        else:
            dataset_new.extend(np.split(row, found))

    return np.array(dataset_new)
dataset = split(dataset)

In [ ]:
dataset.shape

In [ ]:
# data loader

class LanguageModelDataLoader(DataLoader):
    """
        TODO: Define data loader logic here
    """
    def __init__(self, dataset, batch_size, shuffle=True):
        
        self.shuffle = shuffle
        self.dataset = dataset
        self.batch_size = batch_size


    def __iter__(self):
        # concatenate your articles and build into batches
        if self.shuffle:
            np.random.shuffle(self.dataset)
        for i in range(0, self.dataset.shape[0], self.batch_size):
            sentences = self.dataset[i:i+self.batch_size]
            sentences = sorted(sentences, key=lambda l: l.shape[0], reverse=True)
            yield [torch.LongTensor(s[:-1]).to('cuda') for s in sentences], [torch.LongTensor(s[1:]).to('cuda') for s in sentences]

        
        

In [ ]:
l = LanguageModelDataLoader(dataset, 2)
for x, y in l:
    #print(x, y)
    break

In [ ]:
# model

class LanguageModel(nn.Module):
    """
        TODO: Define your model here
    """
    def __init__(self, vocab_size,embed_size,hidden_size, nlayers, tied, dropout):
        super(LanguageModel, self).__init__()
        self.vocab_size=vocab_size
        self.dropout = dropout
        self.embed_size = embed_size
        self.hidden_size = hidden_size
        self.nlayers=nlayers
        self.tied = tied
        self.embedding = nn.Embedding(vocab_size,embed_size)
        self.rnn = nn.LSTM(input_size = embed_size,hidden_size=hidden_size,num_layers=nlayers,dropout=self.dropout)
        self.scoring = nn.Linear(hidden_size,vocab_size)
        if self.tied:
            self.scoring.weight = self.embedding.weight


    def forward(self, seq_list):
        # Feel free to add extra arguments to forward (like an argument to pass in the hiddens)
        batch_size = len(seq_list)
        lens = [len(s) for s in seq_list] # lens of all lines (already sorted)
        bounds = [0]
        for l in lens:
            bounds.append(bounds[-1]+l) # bounds of all lines in the concatenated sequence
        seq_concat = torch.cat(seq_list) # concatenated sequence
        embed_concat = self.embedding(seq_concat) # concatenated embeddings
        embed_list = [embed_concat[bounds[i]:bounds[i+1]] for i in range(batch_size)] # embeddings per line
        packed_input = rnn.pack_sequence(embed_list) # packed version
        hidden = None
        output_packed,hidden = self.rnn(packed_input,hidden)
        output_padded, _ = rnn.pad_packed_sequence(output_packed) # unpacked output (padded)
        output_flatten = torch.cat([output_padded[:lens[i],i] for i in range(batch_size)]) # concatenated output
        scores_flatten = self.scoring(output_flatten) # concatenated logits
        return scores_flatten, hidden

    


In [ ]:
# model trainer

class LanguageModelTrainer:
    def __init__(self, model, loader, max_epochs=1, run_id='exp'):
        """
            Use this class to train your model
        """
        # feel free to add any other parameters here
        self.model = model
        self.loader = loader
        self.train_losses = []
        self.val_losses = []
        self.predictions = []
        self.predictions_test = []
        self.generated_logits = []
        self.generated = []
        self.generated_logits_test = []
        self.generated_test = []
        self.epochs = 0
        self.max_epochs = max_epochs
        self.run_id = run_id
        
        # TODO: Define your optimizer and criterion here
        self.optimizer = torch.optim.Adam(self.model.parameters())
        self.criterion = nn.CrossEntropyLoss()

    def train(self):
        self.model.train() # set to training mode
        epoch_loss = 0
        num_batches = 0
        for batch_num, (inputs, targets) in enumerate(self.loader):
            epoch_loss += self.train_batch(inputs, targets)
        epoch_loss = epoch_loss / (batch_num + 1)
        self.epochs += 1
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)

    def train_batch(self, inputs, targets):
        """ 
            TODO: Define code for training a single batch of inputs
        
        """
        outputs = model(inputs)
        loss = self.criterion(outputs,torch.cat(targets))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss

    
    def test(self):
        # don't change these
        self.model.eval() # set to eval mode
        predictions = TestLanguageModel.prediction(fixtures_pred['inp'], self.model) # get predictions
        self.predictions.append(predictions)
        print(predictions.shape)
        generated_logits = TestLanguageModel.generation(fixtures_gen, 10, self.model) # generated predictions for 10 words
        generated_logits_test = TestLanguageModel.generation(fixtures_gen_test, 10, self.model)
        nll = test_prediction(predictions, fixtures_pred['out'])
        generated = test_generation(fixtures_gen, generated_logits, vocab)
        generated_test = test_generation(fixtures_gen_test, generated_logits_test, vocab)
        self.val_losses.append(nll)
        
        self.generated.append(generated)
        self.generated_test.append(generated_test)
        self.generated_logits.append(generated_logits)
        self.generated_logits_test.append(generated_logits_test)
        
        # generate predictions for test data
        predictions_test = TestLanguageModel.prediction(fixtures_pred_test['inp'], self.model) # get predictions
        self.predictions_test.append(predictions_test)
            
        print('[VAL]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, nll))
        return nll

    def save(self):
        # don't change these
        model_path = os.path.join('experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
        torch.save({'state_dict': self.model.state_dict()},
            model_path)
        np.save(os.path.join('experiments', self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
        np.save(os.path.join('experiments', self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated_test[-1])


In [ ]:
class TestLanguageModel:
    def prediction(batch_seq, model):
        """
            TODO: write prediction code here
            
            :param inp:
            :return: a np.ndarray of logits
        """
        result = []
        for i in range(batch_seq.shape[0]):
            seq = batch_seq[i]
            seq = torch.LongTensor(seq).to('cuda')
            embed = model.embedding(seq).unsqueeze(1) # L x 1 x E
            hidden = None
            output_lstm, hidden = model.rnn(embed,hidden) # L x 1 x H
            output = output_lstm[-1] # 1 x H
            scores = model.scoring(output) # 1 x V
            scores = torch.squeeze(scores, dim=0)
            result.append(scores.cpu().detach().numpy())
        result = np.array(result)
        return result

        
    def generation(batch_seq, n_words, model):
        """
            TODO: write generation code here

            Generate a sequence of words given a starting sequence.
            :param inp: Initial sequence of words (batch size, length)
            :param forward: number of additional words to generate
            :return: generated words (batch size, forward)
        """        
        generated_words = []
        for i in range(batch_seq.shape[0]):
            batch_words = []
            seq = batch_seq[i]
            seq = torch.LongTensor(seq).to('cuda')
            embed = model.embedding(seq).unsqueeze(1) # L x 1 x E

            hidden = None
            output_lstm, hidden = model.rnn(embed,hidden) # L x 1 x H
            output = output_lstm[-1] # 1 x H
            scores = model.scoring(output) # 1 x V
            _,current_word = torch.max(scores,dim=1) # 1 x 1
            batch_words.append(current_word)
            if n_words > 1:
                for i in range(n_words-1):
                    embed = model.embedding(current_word).unsqueeze(0) # 1 x 1 x E
                    output_lstm, hidden = model.rnn(embed,hidden) # 1 x 1 x H
                    output = output_lstm[0] # 1 x H
                    scores = model.scoring(output) # V
                    _,current_word = torch.max(scores,dim=1) # 1
                    batch_words.append(current_word)
                    #if current_word[0].item()==self.stop: # If end of line
                    #    break
            generated_words.append(batch_words)
        
        result = torch.tensor(np.array(generated_words))
        return result
        

In [ ]:
# TODO: define other hyperparameters here

NUM_EPOCHS = 6
BATCH_SIZE = 4


In [ ]:
run_id = str(int(time.time()))
if not os.path.exists('./experiments'):
    os.mkdir('./experiments')
os.mkdir('./experiments/%s' % run_id)
print("Saving models, predictions, and generated words to ./experiments/%s" % run_id)

In [ ]:
model = LanguageModel(len(vocab), embed_size=128, hidden_size=128, nlayers=4, tied=True, dropout=0.5).to('cuda')
loader = LanguageModelDataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)
trainer = LanguageModelTrainer(model=model, loader=loader, max_epochs=NUM_EPOCHS, run_id=run_id)

In [ ]:
best_nll = 1e30 
for epoch in range(NUM_EPOCHS):
    trainer.train()
    nll = trainer.test()
    if nll < best_nll:
        best_nll = nll
        print("Saving model, predictions and generated output for epoch "+str(epoch)+" with NLL: "+ str(best_nll))
        trainer.save()
    

In [ ]:
# Don't change these
# plot training curves
plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.plot(range(1, trainer.epochs + 1), trainer.val_losses, label='Validation losses')
plt.xlabel('Epochs')
plt.ylabel('NLL')
plt.legend()
plt.show()

In [ ]:
# see generated output
print (trainer.generated[-1]) # get last generated output